In [87]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
import dataframe_image as dfi

In [89]:
covar  = pd.read_table('covar_AMP_PD_old.txt', sep = '\t')
covar['Patient'] = covar['FID'] + '_' + covar['IID'] +'.csv'
covar = covar.astype({'Patient':str})
covar = covar.set_index('Patient')

In [61]:
path = os.getcwd() 
l = os.listdir(path)
ls = [i for i in l if i.endswith('csv')]
df = pd.DataFrame()
for i in ls:
    d = pd.read_csv(i)
    print(f'{i} : {len(d)}')
    df = pd.concat([df, d])
df = df.rename(columns = {'Unnamed: 0': 'Patient'})
#df = df[(df['ClinVar_CLNSIG'] == 'Conflicting_interpretations_of_pathogenicity') == False]
#df = df[(df['ClinVar_CLNSIG'] == 'Conflicting_interpretations_of_pathogenicity&risk_factor') == False]
#df = df[(df['ClinVar_CLNSIG'] == 'Conflicting_interpretations_of_pathogenicity&other') == False]
#df = df[(df['ClinVar_CLNSIG'] == 'Conflicting_interpretations_of_pathogenicity&association') == False]
#df = df[(df['ClinVar_CLNSIG'] == 'Conflicting_interpretations_of_pathogenicity&drug_response') == False]
#df = df[(df['ClinVar_CLNSIG'] == 'Conflicting_interpretations_of_pathogenicity&association&risk_factor') == False]
df = df[df['gnomADg_AF_NFE'] < 0.01]
df = df[df['CADD_PHRED'] > 20]
df = df[df['CAROL'].notna()]
df = df.query('CAROL.str.contains("Deleterious")')
df = df.set_index('Patient')
dfs2 = df.astype({'HGVSc':str})
df2 = dfs2[dfs2['HGVSc'].str.contains('NM_198578.4:c.6055G>A')]
#df2 = dfs2[(dfs2['HGVSc'].str.contains('NM_000157.4:c.1226A>G') == False) & (dfs2['HGVSc'].str.contains('NM_198578.4:c.6055G>A') == False)]
df = df.loc[df2.index]
print(len(df))
aggfuncs = ['unique']
dfs = df.groupby('Patient')[['SYMBOL','ID', 'HGVSc', 'gnomADg_AF_NFE', 'ClinVar_CLNDN']].agg(lambda x: list(x))
#dfs = dfs.astype({'CADD_PHRED':float})


10_filtration_onlyClinVar.csv : 11785
11_filtration_onlyClinVar.csv : 18886
12_filtration_onlyClinVar.csv : 11267
13_filtration_onlyClinVar.csv : 5764
14_filtration_onlyClinVar.csv : 10500
15_filtration_onlyClinVar.csv : 10506
16_filtration_onlyClinVar.csv : 15853
17_filtration_onlyClinVar.csv : 15502
18_filtration_onlyClinVar.csv : 4577
19_filtration_onlyClinVar.csv : 12083
1_filtration_onlyClinVar.csv : 25988
21_filtration_onlyClinVar.csv : 3669
22_filtration_onlyClinVar.csv : 4830
2_filtration_onlyClinVar.csv : 35708
3_filtration_onlyClinVar.csv : 12305
4_filtration_onlyClinVar.csv : 11289
5_filtration_onlyClinVar.csv : 14478
6_filtration_onlyClinVar.csv : 14417
7_filtration_onlyClinVar.csv : 12047
8_filtration_onlyClinVar.csv : 10825
9_filtration_onlyClinVar.csv : 12727
7989


In [63]:
common = dfs.join(covar)
common = common.dropna()
common = common[common['SYMBOL'].notna()]

In [65]:
common = common[common['SYMBOL'].notna()]
common2 = common.astype({'HGVSc':str})
#common2 = common2[(common2['HGVSc'].str.contains('NM_000157.4:c.1226A>G') == False) & (common2['HGVSc'].str.contains('NM_198578.4:c.6055G>A') == False)]
common2 = common2[(common2['HGVSc'].str.contains('NM_198578.4:c.6055G>A') == True)]
common = common.loc[common2.index]

In [67]:
common

,SYMBOL,ID,HGVSc,gnomADg_AF_NFE,ClinVar_CLNDN,FID,IID,PAT,MAT,Sex,Status,Age
Patient,,,,,,,,,,,,
BF-1198_BF-1198.csv,"[COL13A1, DGKZ, DYNC2H1, GPD1, LRRK2, PYGL, MY...","[rs200138483, rs76583617, rs140830294, rs20160...","[NM_001368882.1:c.289G>T, NM_001199267.2:c.266...","[0.00020008, 0.00633925, 0.00113485, 0.0, 0.00...","[not_provided&Inborn_genetic_diseases, atypica...",BF-1198,BF-1198,0.0,0.0,1.0,1.0,68.0
HB-PD_INVAE641BA6_HB-PD_INVAE641BA6.csv,"[NODAL, PYGM, MMP20, LRRK2, ABCD4, CREBBP, DNA...","[rs150819707, rs145514333, rs61753770, rs34637...","[NM_018055.5:c.904C>T, NM_005609.4:c.182G>A, N...","[0.000932712, 0.000133351, 0.00439941, 0.00033...",[not_specified&not_provided&Holoprosencephaly_...,HB-PD_INVAE641BA6,HB-PD_INVAE641BA6,0.0,0.0,2.0,2.0,63.0
HB-PD_INVAR519XPV_HB-PD_INVAR519XPV.csv,"[PCDH15, LRRK2, HNF1A, OCA2, CEP152, GLIS2, NO...","[rs111033436, rs34637584, rs202039659, rs18004...","[NM_001384140.1:c.1039C>T, NM_198578.4:c.6055G...","[0.00399893, 0.000333244, 6.69523e-05, 0.0, 0....",[not_specified&not_provided&Usher_syndrome_typ...,HB-PD_INVAR519XPV,HB-PD_INVAR519XPV,0.0,0.0,2.0,2.0,59.0
HB-PD_INVCV174KCK_HB-PD_INVCV174KCK.csv,"[VWF, LRRK2, ACADS, HIF1A, CREBBP, SLC5A2, GDF...","[rs61750603, rs34637584, rs61732144, rs6175570...","[NM_000552.5:c.5191T>A, NM_198578.4:c.6055G>A,...","[0.00059976, 0.000333244, 0.0, 0.00239872, 0.0...",[not_specified&von_Willebrand_disease_type_2&v...,HB-PD_INVCV174KCK,HB-PD_INVCV174KCK,0.0,0.0,2.0,2.0,72.0
HB-PD_INVEB297LZR_HB-PD_INVEB297LZR.csv,"[AAAS, LRRK2, NUBPL, SPTBN4, ACP4, CHIT1, HMCN...","[rs112579822, rs34637584, rs61752327, rs138472...","[NM_015665.6:c.1301G>A, NM_198578.4:c.6055G>A,...","[0.000335435, 0.000333244, 0.00559478, 0.0, 6....",[not_provided&Glucocorticoid_deficiency_with_a...,HB-PD_INVEB297LZR,HB-PD_INVEB297LZR,0.0,0.0,1.0,2.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...
PP-75484_PP-75484.csv,"[LRRK2, COL4A1, CNGB1, CACNA1G, CARD14, DYSF, ...","[rs34637584, rs34004222, rs148999583, rs200980...","[NM_198578.4:c.6055G>A, NM_001845.6:c.161C>T, ...","[0.000333244, 0.0040032, 0.0044047, 0.00093545...",[not_provided&Autosomal_dominant_Parkinson_dis...,PP-75484,PP-75484,0.0,0.0,1.0,2.0,57.0
PP-75505_PP-75505.csv,"[VWF, LRRK2, SCARB1, DNAH17, AGL, APP, HPS4, D...","[rs1800386, rs34637584, rs74830677, rs76449350...","[NM_000552.5:c.4751A>G, NM_198578.4:c.6055G>A,...","[0.00320941, 0.000333244, 6.67289e-05, 0.00306...",[not_provided&von_Willebrand_disease_type_2&vo...,PP-75505,PP-75505,0.0,0.0,1.0,2.0,64.0
PP-75524_PP-75524.csv,"[ATM, LRRK2, KCNH5, NTHL1, MYH14, LYST, COL6A3...","[rs1801673, rs34637584, rs143775193, rs1805378...","[NM_000051.4:c.5558A>T, NM_198578.4:c.6055G>A,...","[0.00792699, 0.000333244, 6.66045e-05, 0.00120...",[Hereditary_cancer-predisposing_syndrome&Famil...,PP-75524,PP-75524,0.0,0.0,1.0,2.0,54.0


In [69]:
common[common['Status'] == 1]

,SYMBOL,ID,HGVSc,gnomADg_AF_NFE,ClinVar_CLNDN,FID,IID,PAT,MAT,Sex,Status,Age
Patient,,,,,,,,,,,,
BF-1198_BF-1198.csv,"[COL13A1, DGKZ, DYNC2H1, GPD1, LRRK2, PYGL, MY...","[rs200138483, rs76583617, rs140830294, rs20160...","[NM_001368882.1:c.289G>T, NM_001199267.2:c.266...","[0.00020008, 0.00633925, 0.00113485, 0.0, 0.00...","[not_provided&Inborn_genetic_diseases, atypica...",BF-1198,BF-1198,0.0,0.0,1.0,1.0,68.0
HB-PD_INVTH110CRQ_HB-PD_INVTH110CRQ.csv,"[PCDH15, FERMT3, LRRK2, SLITRK6, PADI3, NEB, P...","[rs45483395, rs149000560, rs34637584, rs200767...","[NM_001384140.1:c.2885G>T, NM_031471.6:c.130G>...","[0.0026, 0.00560598, 0.000333244, 0.000466978,...",[not_specified&not_provided&Usher_syndrome_typ...,HB-PD_INVTH110CRQ,HB-PD_INVTH110CRQ,0.0,0.0,2.0,1.0,61.0
HB-PD_INVYL203PWY_HB-PD_INVYL203PWY.csv,"[ATM, LRRK2, BLM, NTHL1, LAMA3, POLD1, B3GALNT...","[rs138327406, rs34637584, rs1801256, rs1481044...","[NM_000051.4:c.4388T>G, NM_198578.4:c.6055G>A,...","[0.000399734, 0.000333244, 0.000666223, 0.0013...",[Hereditary_cancer-predisposing_syndrome&not_s...,HB-PD_INVYL203PWY,HB-PD_INVYL203PWY,0.0,0.0,2.0,1.0,63.0
HB-PD_INVZK479NCV_HB-PD_INVZK479NCV.csv,"[LRRK2, LIG4, NTRK1, COL27A1, MPDZ]","[rs34637584, rs72660870, rs144901788, rs141452...","[NM_198578.4:c.6055G>A, NM_206937.2:c.2525C>A,...","[0.000333244, 0.00233209, 0.000602087, 0.00020...",[not_provided&Autosomal_dominant_Parkinson_dis...,HB-PD_INVZK479NCV,HB-PD_INVZK479NCV,0.0,0.0,2.0,1.0,55.0
LB-06417_LB-06417.csv,"[CTNNA3, RELT, LRRK2, LIPC, MC1R, LOXHD1, CR2,...","[rs187752783, rs139368769, rs34637584, rs11329...","[NM_013266.4:c.1132C>T, NM_152222.2:c.1264C>T,...","[0.00140262, 0.000333645, 0.000333244, 0.00733...",[Arrhythmogenic_right_ventricular_dysplasia_13...,LB-06417,LB-06417,0.0,0.0,2.0,1.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...
PP-75420_PP-75420.csv,"[OAT, TCIRG1, RELT, ATM, LRRK2, TBC1D24, UNC13...","[rs148433929, rs144775787, rs139368769, rs1507...","[NM_000274.4:c.478G>A, NM_006019.4:c.2333C>T, ...","[0.000133529, 0.0, 0.000333645, 0.0, 0.0003332...","[Ornithine_aminotransferase_deficiency, Inborn...",PP-75420,PP-75420,0.0,0.0,2.0,1.0,75.0
PP-75422_PP-75422.csv,"[BMPR1A, PDE6C, LRRK2, NUBPL, SPTB, CLN3, MLYC...","[rs140592056, rs148661165, rs34637584, rs35867...","[NM_004329.3:c.1243G>A, NM_006204.4:c.2501A>G,...","[0.000266525, 0.00180048, 0.000333244, 0.00039...",[Juvenile_polyposis_syndrome&Gastrointestinal_...,PP-75422,PP-75422,0.0,0.0,1.0,1.0,73.0
PP-75423_PP-75423.csv,"[DYNC2H1, ATM, C1S, LRRK2, OCA2, SLC5A2, MYO5B...","[rs201825699, rs56009889, rs117907409, rs34637...","[NM_001377.3:c.7462G>A, NM_000051.4:c.6919C>T,...","[0.000133458, 0.000266525, 0.00386255, 0.00033...",[Jeune_thoracic_dystrophy&Short_rib-polydactyl...,PP-75423,PP-75423,0.0,0.0,1.0,1.0,72.0


In [71]:
common[common['Status'] == 2]

,SYMBOL,ID,HGVSc,gnomADg_AF_NFE,ClinVar_CLNDN,FID,IID,PAT,MAT,Sex,Status,Age
Patient,,,,,,,,,,,,
HB-PD_INVAE641BA6_HB-PD_INVAE641BA6.csv,"[NODAL, PYGM, MMP20, LRRK2, ABCD4, CREBBP, DNA...","[rs150819707, rs145514333, rs61753770, rs34637...","[NM_018055.5:c.904C>T, NM_005609.4:c.182G>A, N...","[0.000932712, 0.000133351, 0.00439941, 0.00033...",[not_specified&not_provided&Holoprosencephaly_...,HB-PD_INVAE641BA6,HB-PD_INVAE641BA6,0.0,0.0,2.0,2.0,63.0
HB-PD_INVAR519XPV_HB-PD_INVAR519XPV.csv,"[PCDH15, LRRK2, HNF1A, OCA2, CEP152, GLIS2, NO...","[rs111033436, rs34637584, rs202039659, rs18004...","[NM_001384140.1:c.1039C>T, NM_198578.4:c.6055G...","[0.00399893, 0.000333244, 6.69523e-05, 0.0, 0....",[not_specified&not_provided&Usher_syndrome_typ...,HB-PD_INVAR519XPV,HB-PD_INVAR519XPV,0.0,0.0,2.0,2.0,59.0
HB-PD_INVCV174KCK_HB-PD_INVCV174KCK.csv,"[VWF, LRRK2, ACADS, HIF1A, CREBBP, SLC5A2, GDF...","[rs61750603, rs34637584, rs61732144, rs6175570...","[NM_000552.5:c.5191T>A, NM_198578.4:c.6055G>A,...","[0.00059976, 0.000333244, 0.0, 0.00239872, 0.0...",[not_specified&von_Willebrand_disease_type_2&v...,HB-PD_INVCV174KCK,HB-PD_INVCV174KCK,0.0,0.0,2.0,2.0,72.0
HB-PD_INVEB297LZR_HB-PD_INVEB297LZR.csv,"[AAAS, LRRK2, NUBPL, SPTBN4, ACP4, CHIT1, HMCN...","[rs112579822, rs34637584, rs61752327, rs138472...","[NM_015665.6:c.1301G>A, NM_198578.4:c.6055G>A,...","[0.000335435, 0.000333244, 0.00559478, 0.0, 6....",[not_provided&Glucocorticoid_deficiency_with_a...,HB-PD_INVEB297LZR,HB-PD_INVEB297LZR,0.0,0.0,1.0,2.0,65.0
HB-PD_INVFE237EN8_HB-PD_INVFE237EN8.csv,"[PYGM, DYNC2H1, LRRK2, IFT81, FREM2, ASPM, ITG...","[rs145514333, rs200614421, rs34637584, rs20026...","[NM_005609.4:c.182G>A, NM_001377.3:c.7967G>A, ...","[0.000133351, 6.66756e-05, 0.000333244, 0.0, 0...",[not_specified&not_provided&Glycogen_storage_d...,HB-PD_INVFE237EN8,HB-PD_INVFE237EN8,0.0,0.0,2.0,2.0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...
PP-75484_PP-75484.csv,"[LRRK2, COL4A1, CNGB1, CACNA1G, CARD14, DYSF, ...","[rs34637584, rs34004222, rs148999583, rs200980...","[NM_198578.4:c.6055G>A, NM_001845.6:c.161C>T, ...","[0.000333244, 0.0040032, 0.0044047, 0.00093545...",[not_provided&Autosomal_dominant_Parkinson_dis...,PP-75484,PP-75484,0.0,0.0,1.0,2.0,57.0
PP-75505_PP-75505.csv,"[VWF, LRRK2, SCARB1, DNAH17, AGL, APP, HPS4, D...","[rs1800386, rs34637584, rs74830677, rs76449350...","[NM_000552.5:c.4751A>G, NM_198578.4:c.6055G>A,...","[0.00320941, 0.000333244, 6.67289e-05, 0.00306...",[not_provided&von_Willebrand_disease_type_2&vo...,PP-75505,PP-75505,0.0,0.0,1.0,2.0,64.0
PP-75524_PP-75524.csv,"[ATM, LRRK2, KCNH5, NTHL1, MYH14, LYST, COL6A3...","[rs1801673, rs34637584, rs143775193, rs1805378...","[NM_000051.4:c.5558A>T, NM_198578.4:c.6055G>A,...","[0.00792699, 0.000333244, 6.66045e-05, 0.00120...",[Hereditary_cancer-predisposing_syndrome&Famil...,PP-75524,PP-75524,0.0,0.0,1.0,2.0,54.0


In [73]:
genes = []
for i in common.SYMBOL.to_list():
    for j in i:
        genes.append(j)

genes = np.array(genes)
genes = np.setdiff1d(genes, ['nan'])
genes = list(set(genes))
genes = sorted(genes)

In [75]:
import re
def find_indices(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]
def add_column(common, gene):
    common[gene] = np.nan
    for i in common.index:
        try:
            s = []
            for t in find_indices(common.loc[i, 'SYMBOL'], gene):
                s.append(common.loc[i, 'HGVSc'][t])
            if s!=[]:
                common.loc[i, gene] = str(s)
        except:
            continue
    return common
proba = common
for gene in genes:
    proba = add_column(proba, gene)
result = proba.drop(['SYMBOL', 'ID', 'HGVSc', 'gnomADg_AF_NFE'], axis = 1)
result.dropna(axis='columns',how='all', inplace=True)
def snips_list(gene):
    prelist = result[f'{gene}'].unique()
    l = re.findall(r"N\S+[AGTC]", str(prelist)) # S - непробельный символ
    l = set(l)
    l = list(l)
    return l


In [76]:
len(genes)

947

In [77]:
len(result.columns)

955

In [53]:
genes

['A2ML1',
 'AAAS',
 'AARS2',
 'ABCA1',
 'ABCA12',
 'ABCA3',
 'ABCA4',
 'ABCB11',
 'ABCB4',
 'ABCB6',
 'ABCC6',
 'ABCC8',
 'ABCD4',
 'ABCG5',
 'ABRAXAS1',
 'ACADL',
 'ACADM',
 'ACADS',
 'ACADSB',
 'ACAN',
 'ACD',
 'ACE',
 'ACO2',
 'ACP4',
 'ACP5',
 'ACSF3',
 'ACTN2',
 'ACTN4',
 'ACY1',
 'ADAMTS13',
 'ADAMTS17',
 'ADAMTS18',
 'ADAMTSL4',
 'ADCY5',
 'ADCY6',
 'ADGRG1',
 'ADGRV1',
 'AGA',
 'AGER',
 'AGL',
 'AGPAT2',
 'AGRN',
 'AGT',
 'AGXT',
 'AHI1',
 'AIPL1',
 'AK2',
 'AKAP9',
 'ALB',
 'ALDH3A2',
 'ALDH5A1',
 'ALDH6A1',
 'ALDOB',
 'ALG1',
 'ALG9',
 'ALMS1',
 'ALPK3',
 'ALS2',
 'AMPD1',
 'ANG',
 'ANK1',
 'ANK2',
 'ANK3',
 'ANKRD1',
 'ANKRD31',
 'ANKS6',
 'ANO5',
 'AOC1',
 'AP3B1',
 'AP4M1',
 'APC',
 'APOB',
 'APOC3',
 'APP',
 'ARMC9',
 'ASL',
 'ASPA',
 'ASPM',
 'ATAD3A',
 'ATG4D',
 'ATM',
 'ATN1',
 'ATP13A2',
 'ATP2B4',
 'ATP2C2',
 'ATP7B',
 'ATP8B1',
 'ATR',
 'AUH',
 'AXIN2',
 'B3GALNT2',
 'B3GALT6',
 'B9D1',
 'BAG3',
 'BARD1',
 'BBS12',
 'BBS2',
 'BBS5',
 'BBS7',
 'BBS9',
 'BCHE',
 'BCKD

In [81]:
import pandas as pd

# Preprocess DataFrame to extract necessary data
cases = result[result['Status'] == 2]
control = result[result['Status'] == 1]


# Iterate over pairs of genes
for i in range(len(genes)):
    l = snips_list(genes[i])
    for snip in l:
        # Perform queries only once
        cases_mut1 = cases[cases[genes[i]].str.contains(snip) == True].shape[0]
        control_mut1 = control[control[genes[i]].str.contains(snip) == True].shape[0]
        MAF_cases_mut1 = cases_mut1 / (cases.shape[0] * 2)
        MAF_control_mut1 = control_mut1 / (control.shape[0] * 2)
        if cases_mut1 != 0 or control_mut1 != 0:
            ans0 = pd.DataFrame([{'Mutation1': f'{genes[i]} {snip}',
                    'N cases Mutation1': cases_mut1,
                    'N control Mutation1': control_mut1,
                    'MAF in cases Mutation1': MAF_cases_mut1,
                    'MAF in control Mutation1': MAF_control_mut1,
               }])

                # Write to CSV
            ans0.to_csv(f'LRRK_unc/{genes[i]}.csv', mode='a', header=False, index=False)
            ans0 = None
        else:
            continue

# FROM the check - list

In [21]:
etalon = ['ASH1L','ASXL3', 'ATP13A2','BAG3', 'BIN3', 'BRIP1', 'BST1', 'CAB39L','CAMK2D', 'CASC16', 'CD19', 'CHCHD2', 'CHRNB1', 'CLCN3',
 'CRLS1',
 'CTSB',
 'CYRIB',
 'DJ-1',
 'DLG2',
 'DNAH17',
 'DNAJC13',
 'DNAJC6',
 'DYRK1A',
 'EHMT2-AS1',
 'EIF4G1',
 'ELOVL7',
 'ENSG00000234500',
 'ENSG00000251095',
 'ENSG00000253496',
 'ENSG00000257239',
 'ENSG00000259675',
 'ENSG00000270892',
 'ENSG00000273734',
 'ENSG00000283563',
 'ENSG00000286074',
 'ENSG00000286235',
 'ENSG00000287401',
 'ENSG00000288563',
 'ENSG00000291136',
 'FAM171A2',
 'FAM47E',
 'FAM47E-STBD1',
 'FBXO7',
 'FLG-AS1',
 'GAK',
 'GBA',
 'GBA1',
 'GBF1',
 'GCH1',
 'GIGYF2',
 'GPNMB',
 'GXYLT1',
 'HIP1R',
 'HRNR',
 'HTRA2',
 'IGSF9B',
 'INPP5F',
 'IP6K2',
 'ITGA8',
 'ITPKB',
 'KANSL1',
 'KCNIP3',
 'KCNS3',
 'KPNA1',
 'LCORL',
 'LINC02210-CRHR1',
 'LRRK2',
 'LRRK2-DT',
 'MAP4K4',
 'MAPT-AS1',
 'MCCC1',
 'MED12L',
 'MIPOL1',
 'MUC19',
 'NOD2',
 'NSF',
 'PHB1P21',
 'PINK1',
 'PLA2G6',
 'PMVK',
 'PRKN',
 'PSAP',
 'RAB29',
 'RABEP2',
 'RBMS3',
 'RETREG3',
 'RIT2',
 'RNF141',
 'RPS6KL1',
 'SCARB2',
 'SETD1A',
 'SH3GL2',
 'SIPA1L2',
 'SLC44A4',
 'SNCA',
 'SPPL2B',
 'SPTSSB',
 'SYNJ1',
 'SYT17',
 'TBC1D5',
 'TMEM163',
 'TMEM175',
 'TRIM40',
 'UBAP2',
 'UBTF',
 'UCH-L1',
 'VPS13C',
 'VPS35', 'WNT3']

In [22]:
df_after_check = df_CADD.query(f"SYMBOL in {etalon}")

In [23]:
df_after_check

,Patient,CHROM,POS,ID,SYMBOL,Gene,HGVSc,HGVSp,CADD_PHRED,ClinVar_CLNSIG,ClinVar_CLNDN,LoFtool,CAROL
31,PD-PDKK097AHR_PD-PDKK097AHR.csv,1,155236246,rs75548401,GBA,2629,NM_000157.4:c.1223C>T,NP_000148.2:p.Thr408Met,21.3,Conflicting_interpretations_of_pathogenicity,Inborn_genetic_diseases&not_specified&not_prov...,0.00827,Neutral(0.860)
62,LB-07856_LB-07856.csv,1,155235843,rs76763715,GBA,2629,NM_000157.4:c.1226A>G,NP_000148.2:p.Asn409Ser,24.1,Pathogenic/Likely_pathogenic&risk_factor,Parkinson_disease&Parkinson_disease&_late-onse...,0.00827,Deleterious(0.989)
105,PP-41467_PP-41467.csv,1,155235843,rs76763715,GBA,2629,NM_000157.4:c.1226A>G,NP_000148.2:p.Asn409Ser,24.1,Pathogenic/Likely_pathogenic&risk_factor,Parkinson_disease&Parkinson_disease&_late-onse...,0.00827,Deleterious(0.989)
357,HB-PD_INVMJ225EDX_HB-PD_INVMJ225EDX.csv,1,20645615,rs55831733,PINK1,65018,NM_032409.3:c.1015G>A,NP_115785.1:p.Ala339Thr,24.3,Conflicting_interpretations_of_pathogenicity,not_provided&Autosomal_recessive_early-onset_P...,0.78800,Deleterious(0.999)
467,HB-PD_INVUL535YVX_HB-PD_INVUL535YVX.csv,1,16990276,rs200924194,ATP13A2,23400,NM_022089.4:c.2263C>G,NP_071372.1:p.Gln755Glu,23.1,Conflicting_interpretations_of_pathogenicity,Autosomal_recessive_spastic_paraplegia_type_78...,0.02890,Deleterious(0.999)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201,PP-70714_PP-70714.csv,10,119676794,rs117749531,BAG3,9531,NM_004281.4:c.1240G>A,NP_004272.2:p.Glu414Lys,22.5,Conflicting_interpretations_of_pathogenicity,not_specified&Myofibrillar_myopathy_6&Dilated_...,0.12700,Neutral(0.970)
20685,PP-74977_PP-74977.csv,10,119676794,rs117749531,BAG3,9531,NM_004281.4:c.1240G>A,NP_004272.2:p.Glu414Lys,22.5,Conflicting_interpretations_of_pathogenicity,not_specified&Myofibrillar_myopathy_6&Dilated_...,0.12700,Neutral(0.970)
21044,LB-07204_LB-07204.csv,10,119676794,rs117749531,BAG3,9531,NM_004281.4:c.1240G>A,NP_004272.2:p.Glu414Lys,22.5,Conflicting_interpretations_of_pathogenicity,not_specified&Myofibrillar_myopathy_6&Dilated_...,0.12700,Neutral(0.970)
21726,BF-1092_BF-1092.csv,10,119669950,rs145393807,BAG3,9531,NM_004281.4:c.280A>T,NP_004272.2:p.Ile94Phe,25.7,Conflicting_interpretations_of_pathogenicity,Myofibrillar_myopathy_6&Dilated_cardiomyopathy...,0.12700,Deleterious(0.999)


In [ ]:
aggfuncs = ['unique']
df_group = df_after_check.groupby('Patient')[['SYMBOL', 'ID', 'HGVSc', 'HGVSp', 'ClinVar_CLNDN']].agg(lambda x: list(x))
df_group

,SYMBOL,ID,HGVSc,HGVSp,ClinVar_CLNDN
Patient,,,,,
BF-1002_BF-1002.csv,[GCH1],[rs104894434],[NM_000161.3:c.662T>C],[NP_000152.1:p.Met221Thr],[Dystonia_5&GTP_cyclohydrolase_I_deficiency&no...
BF-1008_BF-1008.csv,"[GBA, BRIP1]","[rs76763715, rs4988345]","[NM_000157.4:c.1226A>G, NM_032043.3:c.517C>T]","[NP_000148.2:p.Asn409Ser, NP_114432.2:p.Arg173...",[Parkinson_disease&Parkinson_disease&_late-ons...
BF-1009_BF-1009.csv,[BAG3],[rs145393807],[NM_004281.4:c.280A>T],[NP_004272.2:p.Ile94Phe],[Myofibrillar_myopathy_6&Dilated_cardiomyopath...
BF-1012_BF-1012.csv,[CHRNB1],[rs201915086],[NM_000747.3:c.305G>C],[NP_000738.2:p.Arg102Pro],[Congenital_myasthenic_syndrome_2A&Congenital_...
BF-1021_BF-1021.csv,"[GBA, NOD2]","[rs78973108, rs2066844]","[NM_000157.4:c.887G>A, NM_001370466.1:c.2023C>T]","[NP_000148.2:p.Arg296Gln, NP_001357395.1:p.Arg...",[Gaucher_disease_type_III&Gaucher_disease_type...
...,...,...,...,...,...
SY-PDTW883TFJ_SY-PDTW883TFJ.csv,[PMVK],[rs373000976],[NM_006556.4:c.79G>T],[NP_006547.1:p.Glu27Ter],[Porokeratosis_1&_Mibelli_type&not_provided&Li...
SY-PDVD904ZLL_SY-PDVD904ZLL.csv,"[MCCC1, NOD2]","[rs148616219, rs2066844]","[NM_020166.5:c.254A>C, NM_001370466.1:c.2023C>T]","[NP_064551.3:p.Asn85Thr, NP_001357395.1:p.Arg6...",[not_provided&3-methylcrotonyl-CoA_carboxylase...
SY-PDWJ270UA9_SY-PDWJ270UA9.csv,[NOD2],[rs61747625],[NM_001370466.1:c.2183C>T],[NP_001357395.1:p.Ala728Val],[Inflammatory_bowel_disease_1&Blau_syndrome&Re...


In [ ]:
df_group.query("@df_group.SYMBOL.str.len() >= 2").to_csv('Results/After_check_ClinVar_result.csv')

In [ ]:
df_two = df_group.query("@df_group.SYMBOL.str.len() >= 2")

In [ ]:
df_two

,SYMBOL,ID,HGVSc,HGVSp,ClinVar_CLNDN
Patient,,,,,
BF-1008_BF-1008.csv,"[GBA, BRIP1]","[rs76763715, rs4988345]","[NM_000157.4:c.1226A>G, NM_032043.3:c.517C>T]","[NP_000148.2:p.Asn409Ser, NP_114432.2:p.Arg173...",[Parkinson_disease&Parkinson_disease&_late-ons...
BF-1021_BF-1021.csv,"[GBA, NOD2]","[rs78973108, rs2066844]","[NM_000157.4:c.887G>A, NM_001370466.1:c.2023C>T]","[NP_000148.2:p.Arg296Gln, NP_001357395.1:p.Arg...",[Gaucher_disease_type_III&Gaucher_disease_type...
HB-PD_INVAH314MA5_HB-PD_INVAH314MA5.csv,"[DNAJC13, NOD2]","[rs61748101, rs2066844]","[NM_015268.4:c.3872A>G, NM_001370466.1:c.2023C>T]","[NP_056083.3:p.Glu1291Gly, NP_001357395.1:p.Ar...","[Parkinson_disease_21&not_provided, Autoinflam..."
HB-PD_INVAK106DV5_HB-PD_INVAK106DV5.csv,"[NOD2, BAG3]","[rs2066845, rs145393807]","[NM_001370466.1:c.2641G>C, NM_004281.4:c.280A>T]","[NP_001357395.1:p.Gly881Arg, NP_004272.2:p.Ile...",[Autoinflammatory_syndrome&Yao_syndrome&Psoria...
HB-PD_INVAR519XPV_HB-PD_INVAR519XPV.csv,"[NOD2, LRRK2]","[rs104895467, rs34637584]","[NM_001370466.1:c.2474A>G, NM_198578.4:c.6055G>A]","[NP_001357395.1:p.Asn825Ser, NP_940980.4:p.Gly...",[Blau_syndrome&Regional_enteritis&Autoinflamma...
...,...,...,...,...,...
PP-75505_PP-75505.csv,"[DNAH17, LRRK2]","[rs76449350, rs34637584]","[NM_173628.4:c.4798G>A, NM_198578.4:c.6055G>A]","[NP_775899.3:p.Val1600Met, NP_940980.4:p.Gly20...","[Inborn_genetic_diseases&not_provided, not_pro..."
PP-75512_PP-75512.csv,"[GBA, NOD2]","[rs76763715, rs104895438]","[NM_000157.4:c.1226A>G, NM_001370466.1:c.1753G>A]","[NP_000148.2:p.Asn409Ser, NP_001357395.1:p.Ala...",[Parkinson_disease&Parkinson_disease&_late-ons...
SY-NIH_INVEP886EEYYL_SY-NIH_INVEP886EEYYL.csv,"[NOD2, LRRK2]","[rs2066845, rs78501232]","[NM_001370466.1:c.2641G>C, NM_198578.4:c.1000G>A]","[NP_001357395.1:p.Gly881Arg, NP_940980.4:p.Glu...",[Autoinflammatory_syndrome&Yao_syndrome&Psoria...
